In [2]:

import oauth2client
from oauth2client.client import GoogleCredentials
from oauth2client import client
from oauth2client.client import flow_from_clientsecrets
from oauth2client import file
from oauth2client import tools
from googleapiclient import discovery
from googleapiclient import errors
from google_auth_oauthlib import flow
import google_auth_httplib2
import httplib2
import time
import os
import io
from googleapiclient.http import MediaIoBaseDownload
import pandas as pd

In [3]:
OAUTH_SCOPES = ("https://www.googleapis.com/auth/dfareporting " + "https://www.googleapis.com/auth/dfatrafficking " + "https://www.googleapis.com/auth/ddmconversions ")

In [4]:
# Set up a Flow object to be used if we need to authenticate.
flow = client.flow_from_clientsecrets(
    "/Users/hamza.ahmed/Coding Projects/Python Apps Authentications/Campaign Manager/client_secret.json", scope = OAUTH_SCOPES, 
    redirect_uri = "urn:ietf:wg:oauth:2.0:oob")

storage = oauth2client.file.Storage('/Users/hamza.ahmed/Coding Projects/Python Apps Authentications/Campaign Manager/credentials.dat')
credentials = storage.get()

if credentials is None or credentials.invalid:
  credentials = tools.run_flow(flow, storage,
                               tools.argparser.parse_known_args()[0])
http = credentials.authorize(httplib2.Http())

In [7]:
service = discovery.build('dfareporting', 'v3.5', http=http)


In [8]:
request = service.userProfiles().list()
response = request.execute()

for profile in response['items']:
  print('Found user profile with ID %s and user name "%s".' %
        (profile['profileId'], profile['userName']))

Found user profile with ID 7323797 and user name "hamza.ahmed".


In [9]:
report_ids_ = []
report_names_ = []

# Construct the request.
request = service.reports().list(profileId= 7323797, scope = "ALL")

response = request.execute()

for report in response['items']:
  report_ids_.append(report['id'])
  report_names_.append(report['name'])

In [190]:
response

{'kind': 'dfareporting#reportList',
 'etag': '"nuLINmHOgpDUOqaxbkIa59jD0M4"',
 'items': [{'id': '870632981',
   'ownerProfileId': '6407874',
   'accountId': '510001',
   'subAccountId': '2860194',
   'name': 'JumpCloud',
   'fileName': 'JumpCloud',
   'kind': 'dfareporting#report',
   'type': 'STANDARD',
   'etag': '"7TTycv44BPwUgqC0zvS5ep_fsOI"',
   'lastModifiedTime': '1647388117000',
   'format': 'CSV',
   'criteria': {'dateRange': {'startDate': '2022-03-01',
     'endDate': '2022-03-15',
     'kind': 'dfareporting#dateRange'},
    'dimensions': [{'name': 'campaign',
      'kind': 'dfareporting#sortedDimension'},
     {'name': 'campaignId', 'kind': 'dfareporting#sortedDimension'},
     {'name': 'site', 'kind': 'dfareporting#sortedDimension'},
     {'name': 'placement', 'kind': 'dfareporting#sortedDimension'},
     {'name': 'date', 'kind': 'dfareporting#sortedDimension'}],
    'metricNames': ['impressions', 'clicks', 'clickRate'],
    'dimensionFilters': [{'dimensionName': 'advertise

In [165]:
report_ids_

['870632981',
 '905182599',
 '930116908',
 '872963562',
 '872963704',
 '927501164',
 '924658802',
 '924118096',
 '921909527',
 '921909340']

In [166]:
  report_file = service.files().get(
      reportId=870632981, fileId=3629104741).execute()

In [167]:
report_file

{'kind': 'dfareporting#file',
 'etag': '"a2-Uprn3pSjjHRnjc9_BDC0cXDE"',
 'reportId': '870632981',
 'id': '3629104741',
 'lastModifiedTime': '1647388134000',
 'status': 'REPORT_AVAILABLE',
 'fileName': 'JumpCloud',
 'format': 'CSV',
 'dateRange': {'kind': 'dfareporting#dateRange',
  'startDate': '2022-03-01',
  'endDate': '2022-03-15'},
 'urls': {'browserUrl': 'https://www.google.com/reporting/downloadFile?id=870632981:3629104741',
  'apiUrl': 'https://www.googleapis.com/dfareporting/v3.5/reports/870632981/files/3629104741?alt=media'}}

In [168]:
report_file['id']

'3629104741'

In [169]:
file_ids = []

for file_id in report_ids_:
    report_file = service.reports().run(
        profileId=7323797, reportId=reportsids).execute()
    file_ids.append(report_file['id'])


#report_names_

{'kind': 'dfareporting#file',
 'etag': '"N74VGWoMmB878izVihVbNRSqd3Q"',
 'reportId': '921909340',
 'id': '3630090510',
 'lastModifiedTime': '1647463061000',
 'status': 'REPORT_AVAILABLE',
 'fileName': 'datorama_delivery',
 'format': 'CSV',
 'dateRange': {'kind': 'dfareporting#dateRange',
  'startDate': '2022-02-17',
  'endDate': '2022-02-20'},
 'urls': {'browserUrl': 'https://www.google.com/reporting/downloadFile?id=921909340:3630090510',
  'apiUrl': 'https://www.googleapis.com/dfareporting/v3.5/reports/921909340/files/3630090510?alt=media'}}

In [170]:
file_ids

['3631072207',
 '3631072209',
 '3631072306',
 '3631072210',
 '3631072216',
 '3631072312',
 '3631072314',
 '3631072162',
 '3631072165',
 '3631072222']

In [171]:
# Retrieve the file metadata.
All_report_files = []
for reportids in report_ids_:
   for fileids in file_ids:
      report_files = service.files().get(reportId=reportids, fileId=fileids).execute()
      All_report_files.append(report_files)

In [188]:
All_report_files

[{'kind': 'dfareporting#file',
  'etag': '"YVXIhS5vgWmDwdFgPXcCQquv6oQ"',
  'reportId': '921909340',
  'id': '3631072207',
  'lastModifiedTime': '1647535232000',
  'status': 'REPORT_AVAILABLE',
  'fileName': 'datorama_delivery',
  'format': 'CSV',
  'dateRange': {'kind': 'dfareporting#dateRange',
   'startDate': '2022-02-17',
   'endDate': '2022-02-20'},
  'urls': {'browserUrl': 'https://www.google.com/reporting/downloadFile?id=921909340:3631072207',
   'apiUrl': 'https://www.googleapis.com/dfareporting/v3.5/reports/921909340/files/3631072207?alt=media'}},
 {'kind': 'dfareporting#file',
  'etag': '"_EE7YlmP9ygBc7Y7pum4EVMgK6E"',
  'reportId': '921909340',
  'id': '3631072209',
  'lastModifiedTime': '1647535232000',
  'status': 'REPORT_AVAILABLE',
  'fileName': 'datorama_delivery',
  'format': 'CSV',
  'dateRange': {'kind': 'dfareporting#dateRange',
   'startDate': '2022-02-17',
   'endDate': '2022-02-20'},
  'urls': {'browserUrl': 'https://www.google.com/reporting/downloadFile?id=92190

In [172]:


report_files = service.files().get(reportId=870632981, fileId=3630090510).execute()
report_files


{'kind': 'dfareporting#file',
 'etag': '"N74VGWoMmB878izVihVbNRSqd3Q"',
 'reportId': '921909340',
 'id': '3630090510',
 'lastModifiedTime': '1647463061000',
 'status': 'REPORT_AVAILABLE',
 'fileName': 'datorama_delivery',
 'format': 'CSV',
 'dateRange': {'kind': 'dfareporting#dateRange',
  'startDate': '2022-02-17',
  'endDate': '2022-02-20'},
 'urls': {'browserUrl': 'https://www.google.com/reporting/downloadFile?id=921909340:3630090510',
  'apiUrl': 'https://www.googleapis.com/dfareporting/v3.5/reports/921909340/files/3630090510?alt=media'}}

In [181]:
def download_file(self, file_name):
        request = self.service.files().get(reportId=870632981, fileId=3630090510).execute()
        fh = io.FileIO(file_name, 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            logger.debug("Download %d%%." % int(status.progress() * 100)) 

In [187]:
# Retrieve the file metadata.
report_file = service.files().get(
    reportId=870632981, fileId=3630090510).execute()

if report_file['status'] == 'REPORT_AVAILABLE':
  # Prepare a local file to download the report contents to.
  out_file = io.FileIO("poop.csv", mode='wb')

  # Create a get request.
  request = service.files().get_media(reportId=870632981, fileId=3630090510)

  # Create a media downloader instance.
  # Optional: adjust the chunk size used when downloading the file.
  downloader =MediaIoBaseDownload(
      out_file, request)

  # Execute the get request and download the file.
  download_finished = False
  while download_finished is False:
    _, download_finished = downloader.next_chunk()

  print('File %s downloaded to %s' % (report_file['id'],
                                      os.path.realpath(out_file.name)))

File 3630090510 downloaded to /Users/hamza.ahmed/Python Apps/API Access/Campaign Manager/poop.csv


In [173]:
fh = io.FileIO("file_name.txt", 'wb')
downloader = MediaIoBaseDownload(fh, report_files)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Downloading {} - {}%".format(file.get('name'), int(status.progress() * 100)))
fh.seek(0)
print (fh, file.get('name'), file.get('mimeType'))

AttributeError: 'dict' object has no attribute 'uri'

In [137]:
df = pd.read_csv(fh)


EmptyDataError: No columns to parse from file

In [85]:


if report_file['status'] == 'REPORT_AVAILABLE':
  # Prepare a local file to download the report contents to.
  out_file = open("myfile.txt", "x")

  # Create a get request.
  request = service.files().get_media(reportId=921909340, fileId=3630090255)

  # Create a media downloader instance.
  # Optional: adjust the chunk size used when downloading the file.
  downloader = http.MediaIoBaseDownload(
      out_file, request, chunksize=CHUNK_SIZE)

  # Execute the get request and download the file.
  download_finished = False
  while download_finished is False:
    _, download_finished = downloader.next_chunk()

  print('File %s downloaded to %s' % (report_file['id'],
                                      os.path.realpath(out_file.name)))

NameError: name 'xsaSD' is not defined

In [64]:
# for reportsids in report_ids_:
#     report_file = service.reports().run(
#         profileId=7323797, reportId=reportsids).execute()
#     report_files.append(report_file)

In [36]:
# sleep = 0
# start_time = time.time()
# while True:
#   report_file = service.files().get(
#       reportId=870632981, fileId=3629104741).execute()

#   status = report_file['status']
#   if status == 'REPORT_AVAILABLE':
#     print ('File status is %s, ready to download.' % status)
#   elif status != 'PROCESSING':
#     print ('File status is %s, processing failed.' % status)
#   elif time.time() - start_time > MAX_RETRY_ELAPSED_TIME:
#     print ('File processing deadline exceeded.')

#   sleep = next_sleep_interval(sleep)
#   print ('File status is %s, sleeping for %d seconds.' % (status, sleep))
#   time.sleep(sleep)

File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status is REPORT_AVAILABLE, ready to download.
File status 

KeyboardInterrupt: 

In [291]:
# target = None

# # Construct the request.
# request = service.accountUserProfiles().get(profileId = 6407874, id = 840998902)
# response = request.execute()
# # while True:
# #   response = request.execute()

# #   for report in response['items']:
# #     if is_target_report(report):
# #       target = report
# #       break

# #   if not target and response['items'] and response['nextPageToken']:
# #     request = service.reports().list_next(request, response)
# #   else:
# #     break

HttpError: <HttpError 401 when requesting https://dfareporting.googleapis.com/dfareporting/v3.5/userprofiles/6407874/accountUserProfiles/840998902?alt=json returned "1075 : Failed to authenticate. Google account can not access the user profile/account requested.". Details: "[{'message': '1075 : Failed to authenticate. Google account can not access the user profile/account requested.', 'domain': 'global', 'reason': 'authError', 'location': 'Authorization', 'locationType': 'header'}]">

In [238]:
response

{'kind': 'dfareporting#accountUserProfile',
 'id': '7323797',
 'name': 'hamza.ahmed',
 'email': 'hamza.ahmed@liquidagency.com',
 'accountId': '510001',
 'subaccountId': '2860194',
 'locale': 'en',
 'userRoleId': '5568543',
 'userAccessType': 'NORMAL_USER',
 'active': True,
 'campaignFilter': {'kind': 'dfareporting#objectFilter', 'status': 'ALL'},
 'siteFilter': {'kind': 'dfareporting#objectFilter', 'status': 'ALL'},
 'userRoleFilter': {'kind': 'dfareporting#objectFilter', 'status': 'ALL'},
 'advertiserFilter': {'kind': 'dfareporting#objectFilter', 'status': 'ALL'}}

In [344]:
target = None

request = service.reports().files().list(
    profileId=7323797, reportId=840998902)
response = request.execute()


while True:
  response = request.execute()

  for report_file in response['items']:
    if (report_file):
      target = report_file
      break

  if not target and response['items'] and response['nextPageToken']:
    request = service.reports().files().list_next(request, response)
  else:
    break

{'nextPageToken': '',
 'kind': 'dfareporting#fileList',
 'etag': '"2HtQtgqxqlEoXFoS85b8zmoCz38"',
 'items': []}

In [252]:
access_token = "ya29.A0ARrdaM9Y10xg_Ujey2lqJGUnO9SyO6HhiDl9qAhMOj4nT9Z_uVex2mQkam1m3Z7tSNyHo7eG8j0UbtGoUsgdawOkqSoPJU0FF4XyreCE4_2JwG0E8-Hq3XBoh4QXgz6qP3JwfBg_XvTigLYwfquyTUCUG2Ul"

In [258]:
# Retrieve the file metadata.
report_file = service.files().get(
    reportId=840998902, fileId = 27290885).execute()

if report_file['status'] == 'REPORT_AVAILABLE':
  # Prepare a local file to download the report contents to.
  out_file = io.FileIO(generate_file_name(report_file), mode='wb')

  # Create a get request.
  request = service.files().get_media(reportId=report_id, fileId=file_id)

  # Create a media downloader instance.
  # Optional: adjust the chunk size used when downloading the file.
  downloader = http.MediaIoBaseDownload(
      out_file, request, chunksize=CHUNK_SIZE)

  # Execute the get request and download the file.
  download_finished = False
  while download_finished is False:
    _, download_finished = downloader.next_chunk()

  print('File %s downloaded to %s' % (report_file['id'],
                                      os.path.realpath(out_file.name)))

HttpError: <HttpError 403 when requesting https://dfareporting.googleapis.com/dfareporting/v3.5/reports/927501164/files/27290885?alt=json returned "Forbidden". Details: "[{'message': 'Forbidden', 'domain': 'global', 'reason': 'forbidden'}]">